# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f79a4416fd0>
├── 'a' --> tensor([[-1.1841, -1.0727, -0.2706],
│                   [-0.7110, -1.2453, -0.1447]])
└── 'x' --> <FastTreeValue 0x7f79a4416df0>
    └── 'c' --> tensor([[ 1.0264, -0.6127,  1.1931, -0.3324],
                        [-0.5817, -0.3046,  2.2446, -0.6356],
                        [ 1.3357,  2.5562,  0.7383, -0.4199]])

In [4]:
t.a

tensor([[-1.1841, -1.0727, -0.2706],
        [-0.7110, -1.2453, -0.1447]])

In [5]:
%timeit t.a

71.6 ns ± 0.13 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f79a4416fd0>
├── 'a' --> tensor([[-1.4137,  0.5450,  0.4632],
│                   [-1.0570, -0.3959,  1.7289]])
└── 'x' --> <FastTreeValue 0x7f79a4416df0>
    └── 'c' --> tensor([[ 1.0264, -0.6127,  1.1931, -0.3324],
                        [-0.5817, -0.3046,  2.2446, -0.6356],
                        [ 1.3357,  2.5562,  0.7383, -0.4199]])

In [7]:
%timeit t.a = new_value

76 ns ± 0.0579 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.1841, -1.0727, -0.2706],
               [-0.7110, -1.2453, -0.1447]]),
    x: Batch(
           c: tensor([[ 1.0264, -0.6127,  1.1931, -0.3324],
                      [-0.5817, -0.3046,  2.2446, -0.6356],
                      [ 1.3357,  2.5562,  0.7383, -0.4199]]),
       ),
)

In [10]:
b.a

tensor([[-1.1841, -1.0727, -0.2706],
        [-0.7110, -1.2453, -0.1447]])

In [11]:
%timeit b.a

66.3 ns ± 0.221 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.6123,  0.5127, -1.8918],
               [-0.9100, -0.3810,  0.0678]]),
    x: Batch(
           c: tensor([[ 1.0264, -0.6127,  1.1931, -0.3324],
                      [-0.5817, -0.3046,  2.2446, -0.6356],
                      [ 1.3357,  2.5562,  0.7383, -0.4199]]),
       ),
)

In [13]:
%timeit b.a = new_value

558 ns ± 1.79 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

890 ns ± 1.69 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 32.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

149 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

153 µs ± 1.76 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f79a441ba60>
├── 'a' --> tensor([[[-1.1841, -1.0727, -0.2706],
│                    [-0.7110, -1.2453, -0.1447]],
│           
│                   [[-1.1841, -1.0727, -0.2706],
│                    [-0.7110, -1.2453, -0.1447]],
│           
│                   [[-1.1841, -1.0727, -0.2706],
│                    [-0.7110, -1.2453, -0.1447]],
│           
│                   [[-1.1841, -1.0727, -0.2706],
│                    [-0.7110, -1.2453, -0.1447]],
│           
│                   [[-1.1841, -1.0727, -0.2706],
│                    [-0.7110, -1.2453, -0.1447]],
│           
│                   [[-1.1841, -1.0727, -0.2706],
│                    [-0.7110, -1.2453, -0.1447]],
│           
│                   [[-1.1841, -1.0727, -0.2706],
│                    [-0.7110, -1.2453, -0.1447]],
│           
│                   [[-1.1841, -1.0727, -0.2706],
│                    [-0.7110, -1.2453, -0.1447]]])
└── 'x' --> <FastTreeValue 0x7f79a441bb50>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.3 µs ± 58 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f78c74ad2e0>
├── 'a' --> tensor([[-1.1841, -1.0727, -0.2706],
│                   [-0.7110, -1.2453, -0.1447],
│                   [-1.1841, -1.0727, -0.2706],
│                   [-0.7110, -1.2453, -0.1447],
│                   [-1.1841, -1.0727, -0.2706],
│                   [-0.7110, -1.2453, -0.1447],
│                   [-1.1841, -1.0727, -0.2706],
│                   [-0.7110, -1.2453, -0.1447],
│                   [-1.1841, -1.0727, -0.2706],
│                   [-0.7110, -1.2453, -0.1447],
│                   [-1.1841, -1.0727, -0.2706],
│                   [-0.7110, -1.2453, -0.1447],
│                   [-1.1841, -1.0727, -0.2706],
│                   [-0.7110, -1.2453, -0.1447],
│                   [-1.1841, -1.0727, -0.2706],
│                   [-0.7110, -1.2453, -0.1447]])
└── 'x' --> <FastTreeValue 0x7f78c74ad6d0>
    └── 'c' --> tensor([[ 1.0264, -0.6127,  1.1931, -0.3324],
                        [-0.5817, -0.3046,  2.2446, -0.6356],
                 

In [23]:
%timeit t_cat(trees)

31.3 µs ± 56.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.3 µs ± 66.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.1841, -1.0727, -0.2706],
                [-0.7110, -1.2453, -0.1447]],
       
               [[-1.1841, -1.0727, -0.2706],
                [-0.7110, -1.2453, -0.1447]],
       
               [[-1.1841, -1.0727, -0.2706],
                [-0.7110, -1.2453, -0.1447]],
       
               [[-1.1841, -1.0727, -0.2706],
                [-0.7110, -1.2453, -0.1447]],
       
               [[-1.1841, -1.0727, -0.2706],
                [-0.7110, -1.2453, -0.1447]],
       
               [[-1.1841, -1.0727, -0.2706],
                [-0.7110, -1.2453, -0.1447]],
       
               [[-1.1841, -1.0727, -0.2706],
                [-0.7110, -1.2453, -0.1447]],
       
               [[-1.1841, -1.0727, -0.2706],
                [-0.7110, -1.2453, -0.1447]]]),
    x: Batch(
           c: tensor([[[ 1.0264, -0.6127,  1.1931, -0.3324],
                       [-0.5817, -0.3046,  2.2446, -0.6356],
                       [ 1.3357,  2.5562,  0.7383, -0.4199]],
         

In [26]:
%timeit Batch.stack(batches)

84.4 µs ± 198 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.1841, -1.0727, -0.2706],
               [-0.7110, -1.2453, -0.1447],
               [-1.1841, -1.0727, -0.2706],
               [-0.7110, -1.2453, -0.1447],
               [-1.1841, -1.0727, -0.2706],
               [-0.7110, -1.2453, -0.1447],
               [-1.1841, -1.0727, -0.2706],
               [-0.7110, -1.2453, -0.1447],
               [-1.1841, -1.0727, -0.2706],
               [-0.7110, -1.2453, -0.1447],
               [-1.1841, -1.0727, -0.2706],
               [-0.7110, -1.2453, -0.1447],
               [-1.1841, -1.0727, -0.2706],
               [-0.7110, -1.2453, -0.1447],
               [-1.1841, -1.0727, -0.2706],
               [-0.7110, -1.2453, -0.1447]]),
    x: Batch(
           c: tensor([[ 1.0264, -0.6127,  1.1931, -0.3324],
                      [-0.5817, -0.3046,  2.2446, -0.6356],
                      [ 1.3357,  2.5562,  0.7383, -0.4199],
                      [ 1.0264, -0.6127,  1.1931, -0.3324],
                      [-0.5817, -

In [28]:
%timeit Batch.cat(batches)

155 µs ± 142 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

378 µs ± 6.72 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
